In [9]:
#Reference => sentdex tutorial on Youtube
!pip install gym #already installed

In [10]:
import gym
env = gym.make('CartPole-v0') #for discription https://github.com/openai/gym/wiki/CartPole-v0
env.reset()

array([-0.00778331, -0.03744282, -0.03113949, -0.01015828])

In [11]:
import random
done = False

while not done:
  action = (int(random.random()*10))%2 #action 0->push cart left; action 1->push cart right
  state, reward, done, _ = env.step(action)# state is a tuple of (cart position, cart velocity, pole angle, pole velocity at tip)
  print(state)  
  env.render()

env.close()

#the cart need to balance the pole as much as possible

[-0.00853217 -0.23210467 -0.03134265  0.27253941]
[-0.01317426 -0.03654984 -0.02589186 -0.02986201]
[-0.01390526  0.15893366 -0.0264891  -0.33060034]
[-0.01072658 -0.0358014  -0.03310111 -0.0463871 ]
[-0.01144261 -0.23043345 -0.03402885  0.2356711 ]
[-0.01605128 -0.42505312 -0.02931543  0.51742904]
[-0.02455234 -0.22953093 -0.01896685  0.21565427]
[-0.02914296 -0.42437667 -0.01465377  0.50229438]
[-0.03763049 -0.61928903 -0.00460788  0.79032349]
[-0.05001627 -0.81434741  0.01119859  1.08155324]
[-0.06630322 -1.00961538  0.03282966  1.37772914]
[-0.08649553 -1.20513162  0.06038424  1.68049566]
[-0.11059816 -1.01075909  0.09399415  1.40721043]
[-0.13081334 -1.20691337  0.12213836  1.72773667]
[-0.15495161 -1.40320159  0.15669309  2.05579505]
[-0.18301564 -1.20999108  0.197809    1.81540939]
[-0.20721547 -1.01754453  0.23411718  1.59014609]


## Implementing Q Table

We will be implementing SARSA

![](SARSA.png)

- State contains continous values
- Q Table will contains prohibitively large number of state values
- We need to **dicretize them** in order to create Q Table
- Initially Q Table will contain random values

In [12]:
#display observation space low and high values
print(env.observation_space.high)#[position,cart_velocity,pole_angle,pole_velocity]
print(env.observation_space.low)#[position,cart_velocity,pole_angle,pole_velocity]
print(env.action_space.n)

[4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]
[-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38]
2


In [92]:
#Reference 1 for discretizizng -> https://medium.com/@tuzzer/cart-pole-balancing-with-q-learning-b54c6068d947 
#Reference 2 for discretizing -> https://github.com/deepakkavoor/cartpole-rl/blob/master/cartpole-q_learning.py
import math
env.observation_space.high[1]= 0.5
env.observation_space.low[1]= -0.5
env.observation_space.high[3]= math.radians(50)
env.observation_space.low[3]= -math.radians(50)

## Now we will implement SARSAMAX algorithm to fill the Q Table

In [102]:
import numpy as np
LEARNING_RATE = 0.1 #hyperparameter 
DISCOUNT = 0.95 #hyperparameter; how important are future rewards  
EPISODES = 20000 #hyperparameter
#Test for higher values of EPISODES -> the performance deteriorate
RENDER_EVERY = 1000



#Reference for discretizing=> https://mc.ai/openai-gyms-cart-pole-balancing-using-q-learning/
buckets=(1, 1, 6, 12,) #hyperparameter -> dimensions of Q Table (position,cart_velocity,pole_angle,pole_velocity)
q_table = np.zeros(buckets + (env.action_space.n,))
print(q_table.shape)



#a helper function to discretize continous states as returned by the environment 
def get_discrete_state(state):
    ratios = [(state[i] + abs(env.observation_space.low[i])) / (env.observation_space.high[i] - env.observation_space.low[i]) for i in range(len(state))]
    new_obs = [int(round((buckets[i] - 1) * ratios[i])) for i in range(len(state))]
    new_obs = [min(buckets[i] - 1, max(0, new_obs[i])) for i in range(len(state))]
    return tuple(new_obs)
    

(1, 1, 6, 12, 2)


In [103]:
# Exploration settings
epsilon = 1  # not a constant, qoing to be decayed
START_EPSILON_DECAYING = 1
END_EPSILON_DECAYING = EPISODES//2
epsilon_decay_value = epsilon/(END_EPSILON_DECAYING - START_EPSILON_DECAYING)

In [104]:
for episode in range(EPISODES):
    done = False
    discrete_state = get_discrete_state(env.reset()) #we always start with initial discrete state'
    if episode % RENDER_EVERY == 0:
        render=True
        print(episode)
    else:
        render=False
    steps_per_episode = 0
    while not done:
        
        if render:
            env.render()
            steps_per_episode+=1
            print('Step Count'+str(steps_per_episode))
        
        if np.random.random() > epsilon: #exploitation
            # Get action from Q table
            action = np.argmax(q_table[discrete_state])
        else:#exploration
            # Get random action
            action = np.random.randint(0, env.action_space.n)
        
        state, reward, done, _ = env.step(action)# state is a tuple of (position,cart_velocity,pole_angle,pole_velocity)
        new_discrete_state = get_discrete_state(state)
        
        if not done:
            q_st_at=q_table[discrete_state+(action,)]
            
            #Find out next action At+1
            if np.random.random() > epsilon:#exploitation
                # Get action from Q table
                action_t_plus_1 = np.argmax(q_table[new_discrete_state])
            else:#exploration
                # Get random action
                action_t_plus_1 = np.random.randint(0, env.action_space.n)
            
            q_st_plus_1_at_plus_1 = q_table[new_discrete_state+(action_t_plus_1,)]
            new_q_st_at = q_st_at + LEARNING_RATE * (reward + DISCOUNT * q_st_plus_1_at_plus_1 - q_st_at)
            q_table[discrete_state+(action,)] = new_q_st_at
        
        discrete_state = new_discrete_state
     # Decaying is being done every episode if episode number is within decaying range
    if END_EPSILON_DECAYING >= episode >= START_EPSILON_DECAYING:
        epsilon -= epsilon_decay_value
env.close()

Episode:0 Epsilon: 1 Decay Value: 5.000250012500625e-05
Episode:1000 Epsilon: 0.9499974998750096 Decay Value: 5.000250012500625e-05
Episode:2000 Epsilon: 0.8999949997500192 Decay Value: 5.000250012500625e-05
Episode:3000 Epsilon: 0.8499924996250288 Decay Value: 5.000250012500625e-05
Episode:4000 Epsilon: 0.7999899995000384 Decay Value: 5.000250012500625e-05
Episode:5000 Epsilon: 0.7499874993750479 Decay Value: 5.000250012500625e-05
Episode:6000 Epsilon: 0.6999849992500575 Decay Value: 5.000250012500625e-05
Episode:7000 Epsilon: 0.6499824991250671 Decay Value: 5.000250012500625e-05
Episode:8000 Epsilon: 0.5999799990000767 Decay Value: 5.000250012500625e-05
Episode:9000 Epsilon: 0.5499774988750863 Decay Value: 5.000250012500625e-05
Episode:10000 Epsilon: 0.4999749987500959 Decay Value: 5.000250012500625e-05
Episode:11000 Epsilon: 0.44997249862510547 Decay Value: 5.000250012500625e-05
Episode:12000 Epsilon: 0.39996999850011505 Decay Value: 5.000250012500625e-05
Episode:13000 Epsilon: 0.34

## Sample Run of Agent

In [74]:

done = False
discrete_state = get_discrete_state(env.reset()) #we always start with initial discrete state'
steps=0
while not done:
    steps+=1
    print(steps)
    env.render()
        
    action = np.argmax(q_table[discrete_state])
        
    state, reward, done, _ = env.step(action)# state is a tuple of (position,cart_velocity,pole_angle,pole_velocity)

    discrete_state=get_discrete_state(state)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
